In [1]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [2]:
# Define a dictionary containing Students data 
#data = {'Name': ['Jai', 'Princi', 'Gaurav', 'Anuj'], 
#        'Height': [5.1, 6.2, 5.1, 5.2], 
#        'Qualification': ['Msc', 'MA', 'Msc', 'Msc']} 
  
# Convert the dictionary into DataFrame 
#df = pd.DataFrame(data) 
  
# Declare a list that is to be converted into a column 
#address = ['Delhi', 'Bangalore', 'Chennai', 'Patna'] 
  
# Using 'Address' as the column name 
# and equating it to the list 
#df['Address'] = address 
  
# Observe the result 
#df

In [3]:
#df = pd.DataFrame({'numbers': [1, 2, 3], 'colors': ['red', 'white', 'blue']}, columns=['numbers', 'colors'])

In [4]:
df_tr_fu_filt_delta = pd.read_csv('df_tr_fu_filt_delta.csv')
df_re_fu_filt_delta = pd.read_csv('df_re_fu_filt_delta.csv')

In [5]:
conteos_1d_tr = pd.read_csv('conteos_1d_tr.csv')
conteos_1d_re = pd.read_csv('conteos_1d_re.csv')

In [6]:
conteos_2d_tr = pd.read_csv('conteos_2d_tr.csv')
conteos_2d_re = pd.read_csv('conteos_2d_re.csv')

In [7]:
conteos_3d_tr = pd.read_csv('conteos_3d_tr.csv')
conteos_3d_re = pd.read_csv('conteos_3d_re.csv')

In [8]:
conteos_4d_tr = pd.read_csv('conteos_4d_tr.csv')
conteos_4d_re = pd.read_csv('conteos_4d_re.csv')

In [9]:
conteos_5d_tr = pd.read_csv('conteos_5d_tr.csv')
conteos_5d_re = pd.read_csv('conteos_5d_re.csv')

#### Establezco los valores de inicio de los sorteos para calcular deltas

In [10]:
df_topes = df_tr_fu_filt_delta.iloc[0::1,[0]]
recent = df_topes.max()
oldest = df_topes.min()

#### Se calcula los valores de diferencia entre los diversos sorteos

In [11]:
def calcular_deltas_verticales(df):
    delticas = []
    previo = recent
    for index, row in df.iterrows():
        resta = int(previo) - int(row[0])
        #print("resta : ", resta, " previo ", previo, " Sorteo: ", row[0], )
        delticas.append(resta)
        previo = int(row[0])
    resta = int(previo) - int(oldest)
    delticas.append(str(resta))
    return delticas

In [12]:
def unsplited_tupla(sbs):
    delticas = []
    partido = sbs.split(sep='|') if (sbs.find("|") >= 1) else sbs
    return partido

In [13]:
def calc_media(df):
    acum = 0
    media = 0
    length = len(df) 
    for i in range(length): 
        acum += int(df[i])
    media = acum/length
    return media

In [14]:
def calc_moda(df):
    repetir = 0                                                                         
    for i in df:                                                                              
        aparece = df.count(i)                                                             
        if aparece > repetir:                                                       
            repetir = aparece 
    
    moda = []                                                                               
    for i in df:                                                                              
        aparece = df.count(i)
        #print("aparece ", aparece)
        if aparece == repetir and i not in moda:                                   
            moda.append(i) 
            #print("moda", moda)
            
    return moda

In [15]:
def procesa_conteo_singles(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []
    for x in range (1,44):
        sbs = [str(x)]
        dftx = df[df.n1.isin(sbs) | df.n2.isin(sbs) |df.n3.isin(sbs) | df.n4.isin(sbs) |df.n5.isin(sbs)]
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)        
        #print(sbs, " X", promedio, " L", last, " M", moda," T",  df_tupla)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['M'] = modaa
    dft['T'] = delta
    return dft

In [16]:
def procesa_conteo_dobles(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2))]
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['M'] = modaa
    dft['T'] = delta
    return dft

In [17]:
def procesa_conteo_triples(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        sbs3 = [str(splitedtupla[2])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2)) 
                & (df.n1.isin(sbs3) | df.n2.isin(sbs3) |df.n3.isin(sbs3) | df.n4.isin(sbs3) |df.n5.isin(sbs3)) ]
        #print(dftx)
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['M'] = modaa
    dft['T'] = delta
    return dft

In [18]:
def procesa_conteo_cuartetos(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        sbs3 = [str(splitedtupla[2])]
        sbs4 = [str(splitedtupla[3])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2)) 
                & (df.n1.isin(sbs3) | df.n2.isin(sbs3) |df.n3.isin(sbs3) | df.n4.isin(sbs3) |df.n5.isin(sbs3))
                & (df.n1.isin(sbs4) | df.n2.isin(sbs4) |df.n3.isin(sbs4) | df.n4.isin(sbs4) |df.n5.isin(sbs4))]
        #print(dftx)
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['M'] = modaa
    dft['T'] = delta
    return dft

In [19]:
def procesa_conteo_quintetos(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []    
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        sbs3 = [str(splitedtupla[2])]
        sbs4 = [str(splitedtupla[3])]
        sbs5 = [str(splitedtupla[4])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2)) 
                & (df.n1.isin(sbs3) | df.n2.isin(sbs3) |df.n3.isin(sbs3) | df.n4.isin(sbs3) |df.n5.isin(sbs3))
                & (df.n1.isin(sbs4) | df.n2.isin(sbs4) |df.n3.isin(sbs4) | df.n4.isin(sbs4) |df.n5.isin(sbs4))
                & (df.n1.isin(sbs5) | df.n2.isin(sbs5) |df.n3.isin(sbs5) | df.n4.isin(sbs5) |df.n5.isin(sbs5)) ]
        #print(dftx)
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['M'] = modaa
    dft['T'] = delta
    return dft

In [20]:
df_cs_tr = procesa_conteo_singles(conteos_1d_tr, df_tr_fu_filt_delta)

In [21]:
df_cd_tr = procesa_conteo_dobles(conteos_2d_tr, df_tr_fu_filt_delta)

In [22]:
df_ct_tr = procesa_conteo_triples(conteos_3d_tr, df_tr_fu_filt_delta)

In [23]:
df_cc_tr = procesa_conteo_cuartetos(conteos_4d_tr, df_tr_fu_filt_delta)

In [24]:
df_cq_tr = procesa_conteo_quintetos(conteos_5d_tr, df_tr_fu_filt_delta)

#### Aqui empiezan los llamados a los calculos de revancha

In [25]:
df_cs_re = procesa_conteo_singles(conteos_1d_re, df_re_fu_filt_delta)

In [26]:
df_cd_re = procesa_conteo_dobles(conteos_2d_re, df_re_fu_filt_delta)

In [27]:
df_ct_re = procesa_conteo_triples(conteos_3d_tr, df_tr_fu_filt_delta)

In [28]:
df_cc_re = procesa_conteo_cuartetos(conteos_4d_tr, df_tr_fu_filt_delta)

In [29]:
df_cq_re = procesa_conteo_quintetos(conteos_5d_tr, df_tr_fu_filt_delta)

#### Aqui inicia el guardado de los dataframe del sorteo tradicional

In [30]:
df_cs_tr.to_csv(r'df_cs_tr.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_cd_tr.to_csv(r'df_cd_tr.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_ct_tr.to_csv(r'df_ct_tr.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_cc_tr.to_csv(r'df_cc_tr.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_cq_tr.to_csv(r'df_cq_tr.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')

#### Aqui inicia el guardado de los dataframe del sorteo revancha

In [31]:
df_cs_re.to_csv(r'df_cs_re.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_cd_re.to_csv(r'df_cd_re.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_ct_re.to_csv(r'df_ct_re.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_cc_re.to_csv(r'df_cc_re.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')
df_cq_re.to_csv(r'df_cq_re.csv', header=['conteo','tupla','X', 'L', 'M', 'T'], index=None, sep=',', mode='w')